# Assignment No 18 :- NLP and Naive Bayes 

In [63]:
# Import necessary libraries
#Import libraries
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
plt.rcParams['figure.figsize'] = (10,5)
plt.rcParams['figure.dpi'] = 300
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from wordcloud import WordCloud
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


### 1. Data Exploration and Preprocessing
* Load the "blogs_categories.csv" dataset and perform an exploratory data analysis to understand its structure and content.

In [64]:
# Load dataset from a CSV file
df = pd.read_csv("blogs.csv")
df.head()


,Data,Labels
0,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
1,Newsgroups: alt.atheism\nPath: cantaloupe.srv....,alt.atheism
2,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,alt.atheism
3,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
4,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...,alt.atheism


In [65]:
# Check for duplicated rows in the dataset
df.duplicated()


0       False
1       False
2       False
3       False
4       False
        ...  
1995    False
1996    False
1997    False
1998    False
1999    False
Length: 2000, dtype: bool

* Preprocess the data by cleaning the text (removing punctuation, converting to lowercase, etc.), tokenizing, and removing stopwords.

In [66]:
# Import necessary libraries
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Labels_num'] = le.fit_transform(df['Labels'])


In [67]:
df


,Data,Labels,Labels_num
0,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism,0
1,Newsgroups: alt.atheism\nPath: cantaloupe.srv....,alt.atheism,0
2,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,alt.atheism,0
3,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism,0
4,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...,alt.atheism,0
...,...,...,...
1995,Xref: cantaloupe.srv.cs.cmu.edu talk.abortion:...,talk.religion.misc,19
1996,Xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc,19
1997,Xref: cantaloupe.srv.cs.cmu.edu talk.origins:4...,talk.religion.misc,19
1998,Xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc,19


In [68]:
lemma = WordNetLemmatizer()


In [69]:
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Yogesh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Yogesh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [70]:
corpus = []

for i in range(len(df)):
    msg = re.sub('[^a-zA-Z]',' ',df['Data'][i]) #Clean the text
    msg = msg.lower() #Text normalization
    msg = msg.split() #Word tokenize
    msg = [lemma.lemmatize(i) for i in msg if i not in stopwords.words('english')] #Lemmatization
    msg = ' '.join(msg) # Join sentences
    corpus.append(msg)


In [71]:
corpus


['path cantaloupe srv c cmu edu magnesium club cc cmu edu news sei cmu edu ci ohio state edu zaphod mp ohio state edu howland reston an net agate doc ic ac uk uknet mcsun germany eu net thoth mchp sni de horus ap mchp sni de frank frank uucp frank dwyer newsgroups alt atheism subject islamic genocide date apr gmt organization siemens nixdorf ag line distribution world message id r vej k horus ap mchp sni de reference r qe fido asd sgi com r ubl bd horus ap mchp sni de r ek uo fido asd sgi com nntp posting host ap mchp sni de article r ek uo fido asd sgi com livesey solntze wpd sgi com jon livesey writes article r ubl bd horus ap mchp sni de frank uucp frank dwyer writes article r qe fido asd sgi com livesey solntze wpd sgi com jon livesey writes noting particular society case mainland uk religously motivated murder murder kind say little whether inter religion murder elsewhere religiously motivated allows one conclude nothing inherent religion matter catholicism protestantism motivates

In [72]:
df


,Data,Labels,Labels_num
0,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism,0
1,Newsgroups: alt.atheism\nPath: cantaloupe.srv....,alt.atheism,0
2,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,alt.atheism,0
3,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism,0
4,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...,alt.atheism,0
...,...,...,...
1995,Xref: cantaloupe.srv.cs.cmu.edu talk.abortion:...,talk.religion.misc,19
1996,Xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc,19
1997,Xref: cantaloupe.srv.cs.cmu.edu talk.origins:4...,talk.religion.misc,19
1998,Xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc,19


In [73]:
y = df['Labels_num']
y


0        0
1        0
2        0
3        0
4        0
        ..
1995    19
1996    19
1997    19
1998    19
1999    19
Name: Labels_num, Length: 2000, dtype: int32

In [74]:
# Initialize and apply TF-IDF vectorizer for text features
tf = TfidfVectorizer()

x_data = tf.fit_transform(corpus)
x_data = x_data.toarray()
x_data


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [75]:
# Split the dataset into training and test sets
xtrain,xtest,ytrain,ytest = train_test_split(x_data,y,test_size=0.2, random_state=1)


In [76]:
# Initialize a Naive Bayes classifier for text classification
nb = MultinomialNB()
nb.fit(xtrain,ytrain)
ypred = nb.predict(xtest)
print(f"Train Accuracy: {nb.score(xtrain,ytrain)}\nTest Accuracy: {nb.score(xtest,ytest)}")


Train Accuracy: 0.985625
Test Accuracy: 0.82


3. Sentiment Analysis

In [77]:
# Import necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Load the data
file_path = 'blogs.csv'  # Adjust this path to where your CSV is stored
data = pd.read_csv(file_path)
# Step 1: Text Preprocessing with TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X = tfidf_vectorizer.fit_transform(data['Data'])


In [ ]:
# Split the dataset into training and test sets


# Step 2: K-means Clustering
kmeans = KMeans(n_clusters=3, random_state=42)
data['Cluster'] = kmeans.fit_predict(X)

# Step 3: Assign Pseudo-Labels to Clusters
# Map clusters to sentiments: 0 -> Positive, 1 -> Negative, 2 -> Neutral (or adjust as necessary)
cluster_to_sentiment = {0: 'Positive', 1: 'Negative', 2: 'Neutral'}
data['Sentiment'] = data['Cluster'].map(cluster_to_sentiment)

# Step 4: Split Data for Training and Testing
X_train, X_test, y_train, y_test = train_test_split(X, data['Sentiment'], test_size=0.2, random_state=42)

# Step 5: Train Naive Bayes Classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

# Step 6: Evaluate the Model
y_pred = nb_classifier.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

    Negative       0.82      0.90      0.86       140
     Neutral       0.83      0.86      0.84       105
    Positive       0.86      0.77      0.81       155

    accuracy                           0.84       400
   macro avg       0.84      0.84      0.84       400
weighted avg       0.84      0.84      0.84       400



In [ ]:
data


,Data,Labels,Cluster,Sentiment
0,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism,1,Negative
1,Newsgroups: alt.atheism\nPath: cantaloupe.srv....,alt.atheism,0,Positive
2,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,alt.atheism,1,Negative
3,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism,1,Negative
4,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...,alt.atheism,1,Negative
...,...,...,...,...
1995,Xref: cantaloupe.srv.cs.cmu.edu talk.abortion:...,talk.religion.misc,1,Negative
1996,Xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc,1,Negative
1997,Xref: cantaloupe.srv.cs.cmu.edu talk.origins:4...,talk.religion.misc,0,Positive
1998,Xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc,1,Negative


In [ ]:
# Import necessary libraries
from sklearn.model_selection import GridSearchCV

param_grid = {'alpha': [0.01, 0.1, 1, 10, 100]}
mnb = MultinomialNB()
grid_search = GridSearchCV(mnb, param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
print(best_params)
print(grid_search.best_score_)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
y_pred


{'alpha': 1}
0.8712500000000001


array(['Negative', 'Neutral', 'Negative', 'Positive', 'Neutral',
       'Neutral', 'Positive', 'Negative', 'Negative', 'Negative',
       'Positive', 'Positive', 'Negative', 'Positive', 'Neutral',
       'Neutral', 'Positive', 'Positive', 'Negative', 'Negative',
       'Positive', 'Negative', 'Negative', 'Negative', 'Negative',
       'Negative', 'Negative', 'Positive', 'Positive', 'Negative',
       'Positive', 'Positive', 'Neutral', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Neutral',
       'Neutral', 'Negative', 'Neutral', 'Neutral', 'Positive',
       'Negative', 'Neutral', 'Negative', 'Positive', 'Negative',
       'Positive', 'Neutral', 'Positive', 'Neutral', 'Negative',
       'Negative', 'Negative', 'Negative', 'Negative', 'Neutral',
       'Positive', 'Positive', 'Negative', 'Positive', 'Negative',
       'Negative', 'Positive', 'Positive', 'Neutral', 'Neutral',
       'Negative', 'Negative', 'Negative', 'Positive', 'Positive',
       'Pos


### Summary
In this notebook, we performed natural language processing tasks on a dataset containing text data. We began by importing essential libraries for data handling, preprocessing, visualization, and model training. After loading the dataset, we explored it through basic methods such as checking for duplicates and converting categorical labels to numerical values.

The dataset underwent preprocessing steps like text vectorization using TF-IDF, which captures the importance of words across documents. Finally, we implemented a Naive Bayes classifier for text classification and visualized key insights using word clouds and other plotting techniques.

### Conclusion
This notebook provided a comprehensive pipeline for text classification, demonstrating key preprocessing, feature extraction, and modeling steps. The Naive Bayes classifier is a straightforward yet powerful tool for handling text classification tasks, and visualization techniques like word clouds offer an intuitive understanding of word distributions in the dataset. Future work could involve experimenting with different classifiers and optimizing the preprocessing pipeline to improve accuracy.
